Paper: https://www.researchgate.net/publication/248284447_Using_the_ADAP_Learning_Algorithm_to_Forcast_the_Onset_of_Diabetes_Mellitus

In [1]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import scipy.stats as stats
from scipy.stats import spearmanr
from scipy.stats import expon
from scipy.stats import norm
import math


In [2]:
# Read the data
df = pd.read_csv('data/diabetes.csv')
df.shape

(768, 9)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


We see that some variables return a value of 0 in some patients, e.g., Glucose and BloodPressure. We will eliminate these patients as these values do not make sense.

In [4]:
# Eliminate values of 0 for variables where it does not make sense
df = df[(df['Glucose']>0) & (df['BloodPressure']>0) & (df['BMI']>0)]
df.shape


(724, 9)

In [5]:
# Drop SkinThickness as it only has a very small contribution
df.drop(['SkinThickness'], axis = 1, inplace = True)


### Distributions and rank correlations

In [6]:
# Glucose: normal distribution
mu = df['Glucose'].mean()
variance = df['Glucose'].var()
sigma = math.sqrt(variance)

# BloodPressure: normal distribution
mu = df['BloodPressure'].mean()
variance = df['BloodPressure'].var()
sigma = math.sqrt(variance)

# BMI: normal distribution
mu = df['BMI'].mean()
variance = df['BMI'].var()
sigma = math.sqrt(variance)

# DiabetesPedigreeFunction: normal distribution
mu = df['DiabetesPedigreeFunction'].mean()
variance = df['DiabetesPedigreeFunction'].var()
sigma = math.sqrt(variance)

# Pregnancies: exponential distribution
mu = df['Pregnancies'].min()
variance = df['Pregnancies'].var()
sigma = math.sqrt(variance)

# Insulin: exponential distribution
mu = df['Insulin'].min()
variance = df['Insulin'].var()
sigma = math.sqrt(variance)

# Age: exponential distribution
mu = df['Age'].min()
variance = df['Age'].var()
sigma = math.sqrt(variance)


In [7]:
# Covariance matrix of rank correlations
cov = []

for i in range(7):
    cov_row = []
    
    for j in range(7):
        cov_row.append(spearmanr(df.iloc[:,i],df.iloc[:,j])[0])
    
    cov.append(cov_row)

cov = np.array(cov)
    

In [8]:
#Rename the columns
df = df.rename(columns = {'Pregnancies': 'f0', 'Glucose': 'f1', 'BloodPressure': 'f2',
                          'Insulin': 'f3', 'BMI': 'f4', 'DiabetesPedigreeFunction': 'f5', 'Age': 'f6'})


### XGB Classifier

In [9]:
X = df.drop(['Outcome'], axis=1)
X = np.array(X)
y = df['Outcome']
y = np.array(y)

# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)


In [10]:
# Some fixed values
seed = 123
deepness = 3
trees = 4
lr = 0.1


In [11]:
# Model Creation
clf = GradientBoostingClassifier(max_depth=deepness, learning_rate=lr, n_estimators=trees)
clf.fit(X_train, y_train)


GradientBoostingClassifier(n_estimators=4)

In [12]:
# Predictions
prediction_clf = clf.predict(X_test)


In [13]:
# Result Analysis
print(confusion_matrix(y_test, prediction_clf))
print('\n')
print(classification_report(y_test, prediction_clf))
print('\n')
print('Accuracy Score: ', round(accuracy_score(y_test, prediction_clf), ndigits=2))


[[47  1]
 [17  8]]


              precision    recall  f1-score   support

           0       0.73      0.98      0.84        48
           1       0.89      0.32      0.47        25

    accuracy                           0.75        73
   macro avg       0.81      0.65      0.65        73
weighted avg       0.79      0.75      0.71        73



Accuracy Score:  0.75


### Robustness calculation tests

In [14]:
from Functions import create_box_thresholds
from Functions import boxes_with_labels
from Functions import create_covariance_matrix
from Functions import calculate_robustness


In [15]:
ROB = []


In [17]:
for t in range(X_test.shape[0]):
#for t in range(10):
    # Scale and loc parameters to model the data in the dimensions
    # Pregnancies: exponential distribution
    loc_1 = X_test[t][0]
    variance = df['f0'].var()
    scale_1 = math.sqrt(variance/10)
    
    # Glucose: normal distribution
    loc_2 = X_test[t][1]
    variance = df['f1'].var()
    scale_2 = math.sqrt(variance/10)
    
    # BloodPressure: normal distribution
    loc_3 = X_test[t][2]
    variance = df['f2'].var()
    scale_3 = math.sqrt(variance/10)

    # Insulin: exponential distribution
    loc_4 = X_test[t][3]
    variance = df['f3'].var()
    scale_4 = math.sqrt(variance/10)
    
    # BMI: normal distribution
    loc_5 = X_test[t][4]
    variance = df['f4'].var()
    scale_5 = math.sqrt(variance/10)

    # DiabetesPedigreeFunction: normal distribution
    loc_6 = X_test[t][5]
    variance = df['f5'].var()
    scale_6 = math.sqrt(variance/10)

    # Age: exponential distribution
    loc_7 = X_test[t][6]
    variance = df['f6'].var()
    scale_7 = math.sqrt(variance/10)
    
    # Test Datapoint
    coordinates = np.array([loc_1,loc_2,loc_3,loc_4,loc_5,loc_6,loc_7])
    
    # Classification according to the classifier
    label = clf.predict([coordinates])[0]
    print('Predicted label:', label)

    # Numbers from the XBG classifier
    forest_num_features = []
    forest_node_count = 0
    forest_features = []
    forest_thresholds = []

    for h in range(trees):
        for j in range(len(clf.estimators_[h])):
            estimator = clf.estimators_[h][j]

            # Get the number of features
            tree_num_features = estimator.tree_.n_features
            forest_num_features.append(tree_num_features)

            # Get the number of nodes
            tree_node_count = estimator.tree_.node_count
            forest_node_count += tree_node_count

            # Get the tree features
            tree_features = estimator.tree_.feature
            forest_features = forest_features + list(tree_features)

            # Get the thresholds
            tree_thresholds = estimator.tree_.threshold
            forest_thresholds = forest_thresholds + list(tree_thresholds)

    forest_num_features = max(forest_num_features)
    forest_thresholds = np.array(forest_thresholds)
    forest_features = np.array(forest_features)
    
    # Get the box thresholds
    BOX = create_box_thresholds(forest_num_features, forest_node_count, forest_features, forest_thresholds)

    # Get all the boxes into a fataframe
    boxes = boxes_with_labels(forest_num_features, BOX)

    # Determine the label for each box (here we subtract 0.01 to make sure we get the right label)
    boxes['class'] = clf.predict(1/2*(boxes.values[:,1::2] + boxes.values[:,:-1:2]))

    # Expand the boxes to infinity in both directions
    boxes = boxes.replace(1000000.0, np.inf)
    boxes = boxes.replace(-1000000.0, -np.inf)

    # Only keep the boxes that have the label of the test point
    boxes = boxes[boxes['class'] == label]

    # DB Transformation
    for h in range(boxes.shape[0]):
        for j in range(boxes.shape[1]-1):
            if j == 1 or j == 0:
                boxes.iloc[[h],[j]] = norm.ppf(expon.cdf(boxes.iloc[h][j], loc=loc_1, scale=scale_1), loc=0, scale=1)
            if j == 3 or j == 2:
                boxes.iloc[[h],[j]] = norm.ppf(norm.cdf(boxes.iloc[h][j], loc=loc_2, scale=scale_2), loc=0, scale=1)
            if j == 5 or j == 4:
                boxes.iloc[[h],[j]] = norm.ppf(norm.cdf(boxes.iloc[h][j], loc=loc_3, scale=scale_3), loc=0, scale=1)
            if j == 7 or j == 6:
                boxes.iloc[[h],[j]] = norm.ppf(expon.cdf(boxes.iloc[h][j], loc=loc_4, scale=scale_4), loc=0, scale=1)
            if j == 9 or j == 8:
                boxes.iloc[[h],[j]] = norm.ppf(norm.cdf(boxes.iloc[h][j], loc=loc_5, scale=scale_5), loc=0, scale=1)
            if j == 11 or j == 10:
                boxes.iloc[[h],[j]] = norm.ppf(norm.cdf(boxes.iloc[h][j], loc=loc_6, scale=scale_6), loc=0, scale=1)
            if j == 13 or j == 12:
                boxes.iloc[[h],[j]] = norm.ppf(expon.cdf(boxes.iloc[h][j], loc=loc_7, scale=scale_7), loc=0, scale=1)

    # Delete boxes that have 0 mass
    delete = []

    for h in range(boxes.shape[0]):
        if boxes.iloc[h]['B1'] == boxes.iloc[h]['T1'] or boxes.iloc[h]['B2'] == boxes.iloc[h]['T2'] or boxes.iloc[h]['B3'] == boxes.iloc[h]['T3'] or boxes.iloc[h]['B4'] == boxes.iloc[h]['T4'] or boxes.iloc[h]['B5'] == boxes.iloc[h]['T5'] or boxes.iloc[h]['B6'] == boxes.iloc[h]['T6'] or boxes.iloc[h]['B7'] == boxes.iloc[h]['T7']:
            delete.append(boxes.index[h])

    boxes.drop(delete, axis=0, inplace=True)
    
    # Only keep the boxes that have the label of the test point
    print('Number of boxes:', boxes.shape[0])

    mean_t = [0,0,0,0,0,0,0]
    
    # Translate the rank correlations into the linear correlations
    cov_lin = 2*np.sin(cov*np.pi/6)
    
    # Result
    rob = calculate_robustness(mean_t, cov_lin, boxes)
    
    #if math.isnan(rob) == True:
    #    rob = 1
        
    ROB.append(rob)
    print('Robustness of the datapoint ' + '[' + str(coordinates[0]) + ', ' + str(coordinates[1]) + ', ' + str(coordinates[2]) + ', ' + str(coordinates[3]) + ', ' + str(coordinates[4]) +  ', ' + str(coordinates[5]) +  ', ' + str(coordinates[6]) + ']: ' + str(rob))
    print()
    

Predicted label: 0
Number of boxes: 2262
Robustness of the datapoint [1.0, 92.0, 62.0, 41.0, 19.5, 0.482, 25.0]: 1.000015572582583

Predicted label: 0
Number of boxes: 754
Robustness of the datapoint [0.0, 111.0, 65.0, 0.0, 24.6, 0.66, 31.0]: 0.9999947204161899

Predicted label: 0
Number of boxes: 2604
Robustness of the datapoint [0.0, 94.0, 70.0, 115.0, 43.5, 0.347, 21.0]: 0.9999666078492107

Predicted label: 1
Number of boxes: 28
Robustness of the datapoint [0.0, 180.0, 90.0, 90.0, 36.5, 0.314, 35.0]: 0.9823804766037343

Predicted label: 0
Number of boxes: 868
Robustness of the datapoint [13.0, 106.0, 70.0, 0.0, 34.2, 0.251, 52.0]: 1.000002190257764

Predicted label: 0
Number of boxes: 656
Robustness of the datapoint [9.0, 124.0, 70.0, 402.0, 35.4, 0.282, 34.0]: 0.9998665154836528

Predicted label: 0
Number of boxes: 868
Robustness of the datapoint [4.0, 125.0, 70.0, 122.0, 28.9, 1.144, 45.0]: 0.9997397783540366

Predicted label: 1
Number of boxes: 16
Robustness of the datapoint [9.0

Number of boxes: 656
Robustness of the datapoint [11.0, 120.0, 80.0, 150.0, 42.3, 0.785, 48.0]: 0.9999424331872698

Predicted label: 0
Number of boxes: 868
Robustness of the datapoint [5.0, 109.0, 75.0, 0.0, 36.0, 0.546, 60.0]: 1.0000152005388832

Predicted label: 0
Number of boxes: 2604
Robustness of the datapoint [0.0, 107.0, 76.0, 0.0, 45.3, 0.686, 24.0]: 0.9999943710884261

Predicted label: 0
Number of boxes: 2604
Robustness of the datapoint [1.0, 96.0, 64.0, 87.0, 33.2, 0.289, 21.0]: 1.0000041314534645

Predicted label: 0
Number of boxes: 868
Robustness of the datapoint [3.0, 132.0, 80.0, 0.0, 34.4, 0.402, 44.0]: 0.9960631666510812

Predicted label: 0
Number of boxes: 868
Robustness of the datapoint [4.0, 156.0, 75.0, 0.0, 48.3, 0.238, 32.0]: 0.6402640910315529

Predicted label: 0
Number of boxes: 754
Robustness of the datapoint [8.0, 112.0, 72.0, 0.0, 23.6, 0.84, 58.0]: 0.9999967228377442

Predicted label: 0
Number of boxes: 1704
Robustness of the datapoint [0.0, 134.0, 58.0, 291

In [18]:
ROB

[1.0000224903519417,
 0.9999999017692026,
 1.0000065092266817,
 1.0000029039410483,
 1.000015572582583,
 0.9999947204161899,
 0.9999666078492107,
 0.9823804766037343,
 1.000002190257764,
 0.9998665154836528,
 0.9997397783540366,
 0.9160376778420143,
 0.9999446725615049,
 1.000005993624419,
 0.9999885813172598,
 0.6410811253153712,
 0.5658103533756207,
 0.9995348749730573,
 1.0000055094884872,
 0.9999867930214804,
 0.9999949262703849,
 0.965352231313747,
 0.9999951030505951,
 1.0000116164581123,
 0.9960473014364138,
 1.0000013161837342,
 0.9922309588811716,
 0.9672357514712181,
 0.9991420566369758,
 1.000006389457373,
 0.8300931272966036,
 0.9999972507269052,
 1.0000010988153425,
 1.0000021631148104,
 1.0000225333915436,
 0.737838999542607,
 1.0000084381405712,
 1.0000005127094618,
 0.9601996597149061,
 0.9999888354055617,
 0.9998783996230294,
 0.928157025922273,
 0.9118685326678555,
 0.9999967579273654,
 0.999990960480824,
 0.97638106461747,
 0.9999810658984657,
 0.9337361051462888,
 0

In [ ]:
 0.9823804766037343,
 0.9160376778420143,
 0.6410811253153712,
 0.5658103533756207,
 0.965352231313747,
 0.9672357514712181,
 0.8300931272966036,
 0.737838999542607,
 0.9601996597149061,
 0.928157025922273,
 0.9118685326678555,
 0.97638106461747,
 0.9337361051462888,
 0.8832077282990043,
 0.9175703179871828,
 0.7144373438401547,
 0.6402640910315529,
 0.3448408523791741,
